<h1>Proyecto, Procesamiento de lenguaje natural aplicado al análisis de opinión política para las elecciones presidenciales en Colombia 2022<span class="tocSkip"></span></h1> 

>**Maestría en Analítica de Datos**  
>**Facultad de Ingeniería y Ciencias Básicas.**  
>**Universidad Central  2021 - III**  
>**Integrantes del trabajo:**  
>- Maria Alejandra Castillo Pabon
>- David Alejandro Ballesteros Díaz

# Librerias

In [1]:
import pandas as pd
from pyvis.network import Network
import numpy as np
#import seaborn as sns

# Cargar de datos procesados
Entorno consolidado de los candidatos

In [2]:
df_sigue= pd.read_csv('entorno_consolidado.csv')
df_sigue = df_sigue.where(pd.notnull(df_sigue), None) 
df_sigue = df_sigue[df_sigue['conteo'] >= 2]
sources = df_sigue['usuario']
targets = df_sigue['nombre']
weights = df_sigue['cantidad_seguidores']

Asignación de color de los nodos

In [3]:
df_sigue['Color'] = ["#ff2d00" if s == "Periodista" else "#ff00bd" if s=="Mediocomunicacion" else "#a8327d" 
                     if s=="Politico" else "#d5d9cc" if s =="Religion" else "#1d1e1a" if s=="PEP" else "#5b3500"
                     if s=="Empresa" else "#09d3ec" for s in df_sigue['grupo']] 

Filtro del segundo grafo para usuarios mas representativos (mayor a 1.000.000 de seguidores)

In [90]:
df_sigue2 = df_sigue[df_sigue['cantidad_seguidores'] >= 1000000]

# Parametros

## Parametros grafos 
Diccionario y lista con los datos de los candidatos para realizar el grafó

In [100]:
sergio_fajardo = {'usuario' : 'sergio_fajardo', 'color' : '#1aff00', 'imagen': 'https://cej.org.co/wp-content/uploads/2018/04/voto_fajardo.png'}
petrogustavo = {'usuario' : 'petrogustavo', 'color' : '#f5ef42', 'imagen': 'https://pbs.twimg.com/profile_images/962471262747529216/Uo8lOkRJ_400x400.jpg'}
OIZuluaga = {'usuario' : 'OIZuluaga', 'color' : '#1325ed', 'imagen': 'https://pbs.twimg.com/profile_images/996539767041986561/xSjJrWRJ_400x400.jpg'}
MariaFdaCabal = {'usuario' : 'MariaFdaCabal', 'color' : '#6c00a6', 'imagen': 'https://pbs.twimg.com/profile_images/1443286044380700683/4l8hXcsI_400x400.jpg'}
ingrodolfohdez = {'usuario' : 'ingrodolfohdez', 'color' : '#a64500', 'imagen': 'https://pbs.twimg.com/profile_images/1176174546342338561/P37zgjin_400x400.jpg'}

candidatos = {"sergio_fajardo" : sergio_fajardo,"petrogustavo" : petrogustavo,'OIZuluaga' : OIZuluaga, 'MariaFdaCabal' : MariaFdaCabal,'ingrodolfohdez' : ingrodolfohdez}
usuarios= {'sergio_fajardo','petrogustavo','OIZuluaga','MariaFdaCabal','ingrodolfohdez'}


# Visualización
## Grafo general del entorno

In [118]:
grafo = Network("850px", "95%",notebook=True,heading="Usuarios de twitter en común por candidat")#font_color='white'

grafo.barnes_hut()

candidato = df_sigue['usuario']
sigue = df_sigue['nombre']
cantidad = df_sigue['cantidad_seguidores']
ids=df_sigue['id']
color=df_sigue['Color']

edge_data = zip(candidato, sigue, cantidad,ids,color)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]
    i = e[3]
    c = e[4]
    if src in usuarios :
        grafo.add_node(src,size=200,label=" ",title= src + format(1,',d'),color=candidatos[src]['color'],shape="circularImage",image = candidatos[src]['imagen'])
    else:
        grafo.add_node(src, src, src,size=w)
    if w > 1000000:
        grafo.add_node(dst,label=dst,level=w,value=100,size=150,title=dst +"<br>Cantidad de seguidores:<br>"+format(w,',d'),color=c)        
    else:
        grafo.add_node(dst, label=" ",level=w, title=dst,value=1)
    grafo.add_edge(src, dst, value=w,minium_weight=0)
    neighbor_map = grafo.get_adj_list()

for node in grafo.nodes:
    node['title'] += '<br>Seguido por:<br>-' + '<br>-'.join(neighbor_map[node['id']])
grafo.show('candidatos.html')

In [117]:
grafo = Network("850px", "95%",notebook=True,heading="Usuarios mas importantes de twitter en común por candidatos")#font_color='white'

candidato = df_sigue2['usuario']
sigue = df_sigue2['nombre']
cantidad = df_sigue2['cantidad_seguidores']
ids=df_sigue2['id']
color=df_sigue2['Color']

edge_data = zip(candidato, sigue, cantidad,ids,color)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]
    i = e[3]
    c = e[4]

    if src in usuarios :
        grafo.add_node(src,size=200,label=" ",title= src + format(1,',d'),color=candidatos[src]['color'],shape="circularImage",image = candidatos[src]['imagen'])
    else:
        grafo.add_node(src, src, src,size=100)
    if w > 3000000:
                grafo.add_node(dst,label=dst,level=w, title=dst +"<br>Cantidad de seguidores:<br>"+format(w,',d'),color=c)
                grafo.add_edge(src, dst,value=w,minium_weight=800)
    else:
        grafo.add_node(dst,label=dst,level=w, title=dst + "<br>Cantidad de seguidores:<br>"+format(w,',d'),color=c)
        grafo.add_edge(src, dst)
    grafo.repulsion(100,spring_length=240)
    neighbor_map = grafo.get_adj_list()

for node in grafo.nodes:
    node['title'] += '<br>Seguido por:<br>-' + '<br>-'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])
grafo.show('candidatos2.html')